# Experiment 03-01: OpenCL Brute

In [88]:
import sys
from glob import glob
import numpy as np
import pyopencl

## Helper Functions

In [89]:
def get_first_platform_by_name(name : str) -> pyopencl.Platform:
    """
    Get the first platform with the given name.

    Parameters
    ----------
    name : str
        The name of the platform to get.
    """
    for platform in pyopencl.get_platforms():
        if platform.name == name:
            return platform

    raise ValueError(f"No platform named '{name}'")

In [90]:
def build_program(program_files : list, context : pyopencl.Context,
        build_options=[]) -> pyopencl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    context : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source_code = ''

    for ocl_file in program_files:
        with open(ocl_file, 'r') as opencl_file:
            file_source_code = opencl_file.read()
            program_source_code += '\n' + file_source_code

    program_source = pyopencl.Program(context, program_source_code)
    program = program_source.build(options=build_options)
            
    return program

In [91]:
def run_ocl_kernel(queue, kernel, global_size,
                   input_tuples, output_tuples,
                   local_size = (32,)):
    
    # copying data onto the device
    for (value, buffer) in input_tuples:
        pyopencl.enqueue_copy(queue, src=value, dest=buffer)
    
    # running program on the device
    kernel_arguments  = [buffer for (_,buffer) in input_tuples] 
    kernel_arguments += [buffer for (_,buffer) in output_tuples]
        
    kernel(queue, global_size, local_size, *kernel_arguments)

    # copying data off the device
    for (value, buffer) in output_tuples:
        pyopencl.enqueue_copy(queue, src=buffer, dest=value)
        
    # waiting for everything to finish
    queue.finish()

## Setup the device and context

In [92]:
ocl_platforms = (platform.name 
                 for platform in pyopencl.get_platforms())
print("\n".join(ocl_platforms))

Apple


In [93]:
gpu_platform = get_first_platform_by_name("Apple")
gpu_devices = gpu_platform.get_devices(device_type=pyopencl.device_type.GPU)
gpu_context = pyopencl.Context(devices=gpu_devices)

print(gpu_devices)
print(gpu_context)

[<pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>]
<pyopencl.Context at 0x6000020d8370 on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>>


## Load Hashcat SHA256 kernel

In [94]:
build_options = [
    
]

program_files = [
    'inc_vendor.cl',
    'inc_types.cl',
    'inc_hash_constants.h',
    'inc_hash_functions.cl',
    'inc_hash_sha256.cl'
]

gpu_program = build_program(program_files, gpu_context, build_options)

# show the kernel names
program_kernel_names = gpu_program.get_info(pyopencl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'Apple M1' on 'Apple' at 0x1027f00>:

Compilation failed: 

program_source:3025:3: warning: implicit declaration of function 'SHA256_STEP_S' is invalid in OpenCL
  ROUND_STEP_S (0);
  ^
program_source:3007:5: note: expanded from macro 'ROUND_STEP_S'
    SHA256_STEP_S (SHA256_F0o, SHA256_F1o, a, b, c, d, e, f, g, h, w0_t, k_sha256[i +  0]); \
    ^
program_source:3025:3: error: use of undeclared identifier 'SHA256_F0o'; did you mean 'SHA256C04'?
program_source:3007:20: note: expanded from macro 'ROUND_STEP_S'
    SHA256_STEP_S (SHA256_F0o, SHA256_F1o, a, b, c, d, e, f, g, h, w0_t, k_sha256[i +  0]); \
                   ^
program_source:2447:3: note: 'SHA256C04' declared here
  SHA256C04=0x3956c25b,
  ^
program_source:3025:3: error: use of undeclared identifier 'SHA256_F1o'
  ROUND_STEP_S (0);
  ^
program_sour